# Calculating Proximity to Melb CBD

- need to change api_key to your api

In [2]:
import time
from openrouteservice import Client
import math
import pandas as pd

In [3]:

# Initialize OpenRouteService client
api_key ='5b3ce3597851110001cf6248fdd0ae85071d43598ef26e7a446a4f78' # PLUG IN YOUR API KEY HERE
client = Client(key=api_key)

# Define Melbourne CBD coordinates
melb_cbd = [144.962646, -37.810272]

def calculate_distances_to_cbd(properties, batch_size):
    distances_to_cbd = []
    num_batches = math.ceil(len(properties) / batch_size)

    for i in range(num_batches):
        # Get current batch of properties
        batch = properties[i * batch_size: (i + 1) * batch_size]
        # Create matrix request with properties as sources and Melb CBD as the destination
        locations = batch + [melb_cbd]  # Batch properties + CBD point

        while True:
            try:
                # Perform matrix request
                response = client.distance_matrix(
                    locations=locations,
                    profile='driving-car',
                    metrics=['distance'],
                    sources=list(range(len(batch))),  # Sources: property indices
                    destinations=[len(batch)]  # Destination: index of Melb CBD
                )

                # Extract distances from response (km)
                distances = [response['distances'][j][0] / 1000 for j in range(len(batch))]
                distances_to_cbd.extend(distances)
                break  # Exit the loop if successful

            except Exception as e:
                print(f"Error with batch {i+1}: {e}")
                if "Rate limit exceeded" in str(e):
                    print("Rate limit exceeded. Waiting for 60 seconds...")
                    time.sleep(60)  # Wait for a minute before retrying
                else:
                    distances_to_cbd.extend([None] * len(batch))  # Append None for failed requests
                    break  # Exit the loop on other errors

        time.sleep(2)  # additional delay between batches

    return distances_to_cbd

## Running Function on Rental Properties

- change domain_df to your df

In [ ]:
# Convert the coordinates column to a list of [longitude, latitude] pairs
property_coordinates = domain_df[['longitude', 'latitude']].values.tolist()

# Calculate distances
distances = calculate_distances_to_cbd(property_coordinates, batch_size=50)

# Add the distances as a new column 'dist_cbd' to the original DataFrame
domain_df['dist_cbd'] = distances

# Display the updated DataFrame with the new column
domain_df.head()

**Adding CBD Dist. to Oldlistings Greater Melb Data (before preprocessing)**

In [4]:
read_dir = '../data/raw/oldlisting/greater_melb_properties_unprocessed.csv'
out_dir = '../data/raw/oldlisting/gm_cbd.csv'
ol_df = pd.read_csv(read_dir)

# Convert the coordinates column to a list of [longitude, latitude] pairs
property_coordinates = ol_df[['longitude', 'latitude']].values.tolist()

# Calculate distances
distances = calculate_distances_to_cbd(property_coordinates, batch_size=50)

# Add the distances as a new column 'dist_cbd' to the original DataFrame
ol_df['dist_cbd'] = distances

# Display the updated DataFrame with the new column
ol_df.head()

ol_df.to_csv(out_dir)